In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Read file CSV 
file_path = 'C:/Users/Admin/Desktop/fraud_detection_project/data/processed/clean_credit_card_transactions.csv'
df = pd.read_csv(file_path)

# Load your data into the DataFrame (assuming you already have it loaded as `df`)

# Aggregating the number of total transactions and fraud transactions by location
location_counts = df.groupby(['lat', 'long']).agg(
    total_transactions=('cc_num', 'count'),
    fraud_transactions=('is_fraud', 'sum')
).reset_index()

# Initialize the map centered at the average latitude and longitude
avg_lat = df['lat'].mean()
avg_long = df['long'].mean()
m = folium.Map(location=[avg_lat, avg_long], zoom_start=5)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each location with popups showing transaction details
for index, row in location_counts.iterrows():
    popup_text = f"""
    <b>Location:</b> ({row['lat']}, {row['long']})<br>
    <b>Total Transactions:</b> {row['total_transactions']}<br>
    <b>Fraudulent Transactions:</b> {row['fraud_transactions']}
    """
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=popup_text,
        icon=folium.Icon(color='red' if row['fraud_transactions'] > 0 else 'blue')
    ).add_to(marker_cluster)

# Save and display the map
m.save("C:/Users/Admin/Desktop/fraud_detection_project/reports/maps/transactions_map.html")
m